In [ ]:
#| default_exp assemble

# assemble

> High-level functions to tidy up already parsed data

In [ ]:
#| export
import pathlib

import pandas as pd

import sproc.structure
import sproc.bundle
import sproc.hier
import sproc.io
import sproc.postprocess
import sproc.parse

In [ ]:
from IPython.core.debugger import set_trace

This `import`s are only meant for testing

In [ ]:
import numpy as np

Directory where the zip files are stored

In [ ]:
directory = pathlib.Path.cwd().parent / 'samples'
assert directory.exists()
directory

PosixPath('/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples')

## Parameters

A (sample) file in that directory

In [ ]:
input_file = directory / 'PlataformasAgregadasSinMenores_202201_05-06.zip'
assert input_file.exists()
input_file

PosixPath('/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/PlataformasAgregadasSinMenores_202201_05-06.zip')

It is parsed and turned into a *column-multiindexed* `pd.DataFrame`

In [ ]:
flat_df = sproc.bundle.read_zip(input_file, concatenate=True)
df = sproc.hier.flat_df_to_multiindexed_df(flat_df)
df.head(2)

id  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip                                             file name                                          entry                                                      
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220105_030012.... 0      https://contrataciondelestado.es/sindicacion/P...   
                                                                                                   1      https://contrataciondelestado.es/sindicacion/P...   

                                                                                                                                                    summary  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip                                             file name                                          entry                                                      
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220105_030012.... 0      Id licitación: 2021039438; Órgano de Contratac...   
                                                                                                   1      Id licitación: 2021_2568; Órgano de Contrataci...   

                                                                                                                                                      title  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip                                             file name                                          entry                                                      
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220105_030012.... 0      L'objecte és la contractació del servei de bug...   
                 

The last few columns

In [ ]:
df.columns[-2:]

MultiIndex([('ContractFolderStatus', 'TenderingProcess', ...),
            ('ContractFolderStatus', 'TenderingProcess', ...)],
           )

Deleted contracts are extracted

In [ ]:
deleted_series = sproc.bundle.read_deleted_zip(input_file)
deleted_series.head(2)

zip                                              file name                                            id                                                                                 
PlataformasAgregadasSinMenores_202201_05-06.zip  PlataformasAgregadasSinMenores_20220105_030012.atom  https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/6724977   2022-01-04 00:12:01.376000+00:00
                                                                                                      https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1968315   2022-01-03 23:11:57.567000+00:00
Name: deleted_on, dtype: datetime64[ns, UTC]

## Handling deleted entries

A function to merge together a `pd.DataFrame` with the actual data and a `pd.Series` like the one shown meant to *flag* some entries as deleted. Ultimately, a new a column, `deleted_on`, is added to the former. It can either contain a date or `pd.NaT` if the corresponding entry was not deleted.

In [ ]:
#| export
def merge_deleted(data_df: pd.DataFrame, deleted_series: pd.Series) -> pd.DataFrame:
    
    # duplicates are dropped (by means of `deduplicate_deleted_series`), so is the `file name` index, and the result turned into a dataframe
    deduplicated_deleted_df = sproc.postprocess.deduplicate_deleted_series(deleted_series).droplevel(level=['zip', 'file name']).to_frame()
    
    # if this is a column-multiindexed dataframe
    if type(data_df.columns) == pd.MultiIndex:
    
        # in order to merge this new `pd.DataFrame` with `data_df` we need a *multiindex* for the former with the same number of levels as in the latter
        deduplicated_deleted_df.columns = pd.MultiIndex.from_tuples([sproc.hier.pad_col_levels(data_df, ['deleted_on'])])
    
    # the `data_df` is (*left*-)joined with the new one yielding deleted entries; the result is a dataframe in which a new (flag) column `deleted_on`
    # was added; notice that `data_df`'s index is reset for easying the merge (assuming every contract
    # shows only once in `data_df`, *id* should still provide a unique index...though it probably doesn't matter anyway)
    res = data_df.reset_index().set_index(['id']).merge(deduplicated_deleted_df, how='left', on=['id'])
    
    # on return, the index is left as it was
    return res.reset_index().set_index(['zip', 'file name', 'entry'])

In [ ]:
merged_df = merge_deleted(df, deleted_series)
merged_df.head(2)

id  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip                                             file name                                          entry                                                      
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220105_030012.... 0      https://contrataciondelestado.es/sindicacion/P...   
                                                                                                   1      https://contrataciondelestado.es/sindicacion/P...   

                                                                                                                                                    summary  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip                                             file name                                          entry                                                      
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220105_030012.... 0      Id licitación: 2021039438; Órgano de Contratac...   
                                                                                                   1      Id licitación: 2021_2568; Órgano de Contrataci...   

                                                                                                                                                      title  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip                                             file name                                          entry                                                      
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220105_030012.... 0      L'objecte és la contractació del servei de bug...   
                 

For column-wise *flat* `pd.DataFrame`s

In [ ]:
merge_deleted(flat_df, deleted_series).head(2)

id  \
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20220105_030012.... 0      https://contrataciondelestado.es/sindicacion/P...   
                                                   1      https://contrataciondelestado.es/sindicacion/P...   

                                                                                                      zip  \
file name                                          entry                                                    
PlataformasAgregadasSinMenores_20220105_030012.... 0      PlataformasAgregadasSinMenores_202201_05-06.zip   
                                                   1      PlataformasAgregadasSinMenores_202201_05-06.zip   

                                                                                                    summary  \
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20220105_030012.... 0      Id licitación: 2021039438; Órgano de Contratac...   
                                                   1      Id licitación: 2021_2568; Órgano de Contrataci...   

                                                                                                      title  \
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20220105_030012.... 0      L'objecte és la contractació del servei de bug...   
                                                   1      Servei comunicacions postals de l'Ajuntament d...   

                                                                                  updated  \
file name                                          entry                                    
PlataformasAgregadasSinMenores_20220105_030012.... 0     2022-01-04 12:12:09.464000+00:00   
                                                   1     2022-01-04 12:12:09.400000+00:00   

                                                         ContractFolderStatus - ContractFolderID  \
file name                                          entry                                           
PlataformasAgregadasSinMenores_20220105_030012.... 0                                  2021039438   
                                                   1                                   2021_2568   

                                                         ContractFolderStatus - ContractFolderStatusCode  \
file name                                          entry                                                   
PlataformasAgregadasSinMenores_20220105_030012.... 0                                                 RES   
                                                   1                                                 ADJ   

                                                         ContractFolderStatus - LocatedContractingParty - BuyerProfileURIID  \
file name                                          entry                                                                      
PlataformasAgregadasSinMenores_20220105_030012.... 0      https://contractaciopublica.gencat.cat/ecofin_...                   
                                                   1      https://contractaciopublica.gencat.cat/ecofin_...                   

                                                         ContractFolderStatus - LocatedContractingParty - Party - PartyName - Name  \
file name                                          entry                                                                             
PlataformasAgregadasSinMenores_20220105_030012.... 0                                   Ajuntament de Girona                          
                                                   1                                 Ajuntament de Collbató                          

                                                         ContractFolderStatus - 

The last column is the new one added

In [ ]:
merged_df.iloc[:, -2:]

ContractFolderStatus  \
                                                                       TenderingProcess   
                                                         TenderSubmissionDeadlinePeriod   
                                                                                          
                                                                                          
                                                                                          
file name                                          entry                                  
PlataformasAgregadasSinMenores_20220105_030012.... 0          2021-12-13 23:59:00+00:00   
                                                   1          2021-11-29 23:59:00+00:00   
                                                   2          2021-12-20 14:00:00+00:00   
                                                   3          2021-10-22 14:00:00+00:00   
                                                   4          2021-11-30 18:00:00+00:00   
..                                                                                  ...   
PlataformasAgregadasSinMenores_20220106_030013.... 471        2022-01-19 15:00:00+00:00   
                                                   472        2021-05-07 15:00:00+00:00   
                                                   473        2021-06-30 23:59:00+00:00   
                                                   474        2021-12-13 13:00:00+00:00   
                                                   475        2021-11-10 18:00:00+00:00   

                                                         deleted_on  
                                                                     
                                                                     
                                                                     
                                                                     
                                                                     
file name                                          entry             
PlataformasAgregadasSinMenores_20220105_030012.... 0            NaT  
                                                   1            NaT  
                                                   2            NaT  
                                                   3            NaT  
                                                   4            NaT  
..                                                              ...  
PlataformasAgregadasSinMenores_20220106_030013.... 471          NaT  
                                                   472          NaT  
                                                   473          NaT  
                                                   474          NaT  
                                                   475          NaT  

[951 rows x 2 columns]

The new *merged* `pd.DataFrame` has the same number of rows as the original one

In [ ]:
assert len(df) == len(merged_df)

How many entries have been deleted?

In [ ]:
merged_df['deleted_on'].notna().sum()

0

## Saving to parquet

In order to save a `pd.DataFrame` in a *parquet* file, some conditions must be met:

1. if some column contains a list, then every element in that columns must also be a list (of a single element if necessary)

1. the elements of the lists in any column must be of the same type

1. all of the lists across a given column must have elements of the same type

In [ ]:
#| export
def parquet_amenable(df: pd.DataFrame, inplace: bool = False) -> pd.DataFrame:
    
    if inplace:
        
        res = df
        
    else:
        
        res = df.copy()
        
    # 1: every element in a multivalued column must be a list (maybe with a single element)
    res = sproc.io.homogenize_multivalued(res)
    
    # a list with the names of columns that are *multivalued*
    multivalued_columns = sproc.structure.multivalued_columns(res)
    
    # 2: the same type is enforced in all the `list`s (across rows and columns) in *multivalued* columns
    res[multivalued_columns] = res[multivalued_columns].applymap(sproc.io.cast_list_to_floats_or_strs)
    
    # 3: the same type for the elements in the list is enforced across every **individual** multivalued column
    res[multivalued_columns] = res[multivalued_columns].apply(sproc.io.cast_multivalued_series_to_common_type, axis='index')
    
    return res

In [ ]:
parquet_df = parquet_amenable(df)
parquet_df.head(2)

id  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip                                             file name                                          entry                                                      
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220105_030012.... 0      https://contrataciondelestado.es/sindicacion/P...   
                                                                                                   1      https://contrataciondelestado.es/sindicacion/P...   

                                                                                                                                                    summary  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip                                             file name                                          entry                                                      
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220105_030012.... 0      Id licitación: 2021039438; Órgano de Contratac...   
                                                                                                   1      Id licitación: 2021_2568; Órgano de Contrataci...   

                                                                                                                                                      title  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip                                             file name                                          entry                                                      
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220105_030012.... 0      L'objecte és la contractació del servei de bug...   
                 

## Concatenating dataframes

Concatenating `pd.DataFrame`s with the same number of levels in the columns is fine

In [ ]:
pd.concat((df.iloc[:2], df.iloc[2:4]), axis=0).head(2)

id  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip                                             file name                                          entry                                                      
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220105_030012.... 0      https://contrataciondelestado.es/sindicacion/P...   
                                                                                                   1      https://contrataciondelestado.es/sindicacion/P...   

                                                                                                                                                    summary  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip                                             file name                                          entry                                                      
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220105_030012.... 0      Id licitación: 2021039438; Órgano de Contratac...   
                                                                                                   1      Id licitación: 2021_2568; Órgano de Contrataci...   

                                                                                                                                                      title  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip                                             file name                                          entry                                                      
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220105_030012.... 0      L'objecte és la contractació del servei de bug...   
                 

Let us build a dummy `pd.DataFrame` with *multiindex*es for both rows and columns

In [ ]:
dummy_df = pd.DataFrame(
    np.random.rand(2, 4),
    columns=pd.MultiIndex.from_product([['a', 'b'], range(1, 3)]),
    index=pd.MultiIndex.from_tuples([('file.zip', 'foo.atom', 1), ('file.zip', 'fee.atom', 2)], names=['zip', 'file name', 'entry']))
dummy_df

a                   b          
                                 1         2         1         2
zip      file name entry                                        
file.zip foo.atom  1      0.271482  0.150680  0.946666  0.997345
         fee.atom  2      0.839853  0.243164  0.199644  0.984693

This new dummy `pd.DataFrame` and the one above cannot be stacked together as they are because they have different number of levels in the columns

In [ ]:
try:
    pd.concat((df, dummy_df), axis=0)
except ValueError:
    print(f'{df.columns.nlevels=}, {dummy_df.columns.nlevels=}')

df.columns.nlevels=6, dummy_df.columns.nlevels=2


The resulting `pd.DataFrame` must have the larger number of levels

In [ ]:
res_n_levels = max(df.columns.nlevels, dummy_df.columns.nlevels)
res_n_levels

6

Then we need to complement the names of the columns for the *shallower* `pd.DataFrame`

In [ ]:
new_names = []

for c in dummy_df.columns:
    new_names.append(c + ('',) * (res_n_levels - dummy_df.columns.nlevels))
    
print(new_names)

[('a', 1, '', '', '', ''), ('a', 2, '', '', '', ''), ('b', 1, '', '', '', ''), ('b', 2, '', '', '', '')]


A new *padded* `pd.DataFrame` is built out of the dummy

In [ ]:
padded_dummy_df = dummy_df.copy()
padded_dummy_df.columns = pd.MultiIndex.from_tuples(new_names)
padded_dummy_df.columns.nlevels

6

Now it's fine to concatenate both `pd.DataFrame`s

In [ ]:
pd.concat((df, padded_dummy_df), axis=0).tail(3)

id  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip                                             file name                                          entry                                                      
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220106_030013.... 475    https://contrataciondelestado.es/sindicacion/P...   
file.zip                                        foo.atom                                           1                                                   <NA>   
                                                fee.atom                                           2                                                   <NA>   

                                                                                                                                                    summary  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip                                             file name                                          entry                                                      
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220106_030013.... 475    Id licitación: CONTR/2021/000000137; Órgano de...   
file.zip                                        foo.atom                                           1                                                   <NA>   
                                                fee.atom                                           2                                                   <NA>   

                                                                                                                                                      title  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip              

The above steps are summarized in a new function.

In [ ]:
#| export
def stack(top_df: pd.DataFrame, bottom_df: pd.DataFrame) -> pd.DataFrame:
    
    assert top_df.index.names == bottom_df.index.names, 'DataFrames are expected to have indexes with the same names'
    
    # if we have different number of levels in the dataframes...
    if top_df.columns.nlevels != bottom_df.columns.nlevels:
        
        if top_df.columns.nlevels < bottom_df.columns.nlevels:
            
            to_be_fixed_df = top_df
            
            res_n_levels = bottom_df.columns.nlevels
            
        else:
            
            to_be_fixed_df = bottom_df
            
            res_n_levels = top_df.columns.nlevels
            
        new_names = []
            
        for c in to_be_fixed_df.columns:
            
            new_names.append(c + ('',) * (res_n_levels - to_be_fixed_df.columns.nlevels))
            
        to_be_fixed_df.columns = pd.MultiIndex.from_tuples(new_names)
        
    return pd.concat((top_df, bottom_df), axis=0)

Above `df` and the dummy one are stacked together

In [ ]:
stack(df, dummy_df).tail(3)

id  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip                                             file name                                          entry                                                      
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220106_030013.... 475    https://contrataciondelestado.es/sindicacion/P...   
file.zip                                        foo.atom                                           1                                                   <NA>   
                                                fee.atom                                           2                                                   <NA>   

                                                                                                                                                    summary  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip                                             file name                                          entry                                                      
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220106_030013.... 475    Id licitación: CONTR/2021/000000137; Órgano de...   
file.zip                                        foo.atom                                           1                                                   <NA>   
                                                fee.atom                                           2                                                   <NA>   

                                                                                                                                                      title  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip              

...in a different order

In [ ]:
stack(dummy_df, df).head(3)

a  \
                                                                                                                 1   
                                                                                                                     
                                                                                                                     
                                                                                                                     
                                                                                                                     
zip                                             file name                                          entry             
file.zip                                        foo.atom                                           1      0.271482   
                                                fee.atom                                           2      0.839853   
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220105_030012.... 0           NaN   

                                                                                                                    \
                                                                                                                 2   
                                                                                                                     
                                                                                                                     
                                                                                                                     
                                                                                                                     
zip                                             file name                                          entry             
file.zip                                        foo.atom                                           1      0.150680   
                                                fee.atom                                           2      0.243164   
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220105_030012.... 0           NaN   

                                                                                                                 b  \
                                                                                                                 1   
                                                                                                                     
                                                                                                                     
                                                                                                                     
                                                                                                                     
zip                                             file name                                          entry             
file.zip                                        foo.atom                                           1      0.946666   
                                                fee.atom                                           2      0.199644   
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220105_030012.... 0           NaN   

                                                                                                                    \
                                                                                                                 2   
                                                                                                                     
                                                                                                                     
                                                                                                  

## Processing a zip

A convenience function to streamline a common procedure to process data in *zip* files

In [ ]:
#| export
def distilled_data_from_zip(zip_file: pathlib.Path | str) -> tuple[pd.DataFrame, pd.Series]:
    
    # in case a `str` was passed
    zip_file = pathlib.Path(zip_file)
    
    # data is read from the above *zip* file (every file's data concatenated into a single `pd.DataFrame`)...
    df = sproc.bundle.read_zip(zip_file, concatenate=True)
    
    # ...and re-structured with multiindexed columns
    df = sproc.hier.flat_df_to_multiindexed_df(df)
    
    # the same contract might show up more than once due to updates...but only the last one is kept
    last_update_only_df = sproc.postprocess.keep_updates_only(df)
    
    # the same zip file also contains information (at the beginning) about deleted entries
    deleted_series = sproc.bundle.read_deleted_zip(zip_file)
    
    return last_update_only_df, deleted_series

In [ ]:
distilled_df, distilled_series = distilled_data_from_zip(input_file)

In [ ]:
distilled_df.head(2)

id  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip                                             file name                                          entry                                                      
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220105_030012.... 474    https://contrataciondelestado.es/sindicacion/P...   
                                                                                                   473    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                                                                    summary  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip                                             file name                                          entry                                                      
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220105_030012.... 474    Id licitación: 15/2021; Órgano de Contratació...   
                                                                                                   473    Id licitación: 2021/CO_ASER/0120; Órgano de C...   

                                                                                                                                                      title  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip                                             file name                                          entry                                                      
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220105_030012.... 474    “Reforestación de 8,8 HA en el paraje “MONTAÑO...   
                 

In [ ]:
distilled_series.head(2)

zip                                              file name                                            id                                                                                 
PlataformasAgregadasSinMenores_202201_05-06.zip  PlataformasAgregadasSinMenores_20220105_030012.atom  https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/6724977   2022-01-04 00:12:01.376000+00:00
                                                                                                      https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1968315   2022-01-03 23:11:57.567000+00:00
Name: deleted_on, dtype: datetime64[ns, UTC]

## Completeness

It computes the *filling ratio* of every *identified* administration for every column. *Non-identified* administration will be gathered together in a `NaN` *domain*.

In [ ]:
#| export
def sparsity(df: pd.DataFrame, tidy_up: bool = False, do_not_modify_input: bool = True) -> pd.DataFrame:
    "Ratio of completeness for every (identified) administration"
    
    # a new `Domain` column is added
    df['domain'] = sproc.parse.domain(df)
    
    # it is used for grouping
    domain_grouped_df = df.groupby('domain', dropna=False)
    
    # for the sake of convenience
    domain_col = sproc.hier.pad_col_levels(df, ['domain'])
    
    # for every group we ascertain which elements (across every column) are not null (still a pd.DataFrame),
    # and then summarize it (one value per group and column) with mean (ratio of filled-in values)
    filled_in_ratio_df = domain_grouped_df.apply(
        lambda x: x.notnull().mean()).drop(domain_col, axis=1)
    
    if tidy_up:
        
        filled_in_ratio_df = filled_in_ratio_df.sort_index(axis=1).T
        
    if do_not_modify_input:
    
        # the added column is dropped
        df.drop(domain_col, axis=1, inplace=True)
    
    return filled_in_ratio_df

In [ ]:
sparsity(df)

id summary title updated  \
                                                            
                                                            
                                                            
                                                            
                                                            
domain                                                      
contractaciopublica.gencat.cat  1.0     1.0   1.0     1.0   
www.bilbao.eus                  1.0     1.0   1.0     1.0   
www.contratacion.euskadi.eus    1.0     1.0   1.0     1.0   
www.contratosdegalicia.gal      1.0     1.0   1.0     1.0   
www.larioja.org                 1.0     1.0   1.0     1.0   
www.madrid.org                  1.0     1.0   1.0     1.0   
NaN                             1.0     1.0   1.0     1.0   

                               ContractFolderStatus                           \
                                   ContractFolderID ContractFolderStatusCode   
                                                                               
                                                                               
                                                                               
                                                                               
domain                                                                         
contractaciopublica.gencat.cat                  1.0                      1.0   
www.bilbao.eus                                  1.0                      1.0   
www.contratacion.euskadi.eus                    1.0                      1.0   
www.contratosdegalicia.gal                      1.0                      1.0   
www.larioja.org                                 1.0                      1.0   
www.madrid.org                                  1.0                      1.0   
NaN                                             1.0                      1.0   

                                                                  \
                               LocatedContractingParty             
                                     BuyerProfileURIID     Party   
                                                       PartyName   
                                                            Name   
                                                                   
domain                                                             
contractaciopublica.gencat.cat                     1.0       1.0   
www.bilbao.eus                                     0.0       1.0   
www.contratacion.euskadi.eus                       0.0       1.0   
www.contratosdegalicia.gal                         1.0       1.0   
www.larioja.org                                    1.0       1.0   
www.madrid.org                                     1.0       1.0   
NaN                                                0.0       1.0   

                                                                      ...  \
                                                  ProcurementProject  ...   
                               ParentLocatedParty               Name  ...   
                                        PartyName                     ...   
                                             Name                     ...   
                                                                      ...   
domain                                                                ...   
contractaciopublica.gencat.cat           1.000000                1.0  ...   
www.bilbao.eus                           1.000000                1.0  ...   
www.contratacion.euskadi.eus             1.000000                1.0  ...   
www.contratosdegalicia.gal               0.000000                1.0  ...   
www.larioja.org                          1.000000                1.0  ...   
www.madrid.org                           0.747826                1.0  ...   
NaN                                      1.000000                1.0  ...   

                         

The *tidied-up* version

In [ ]:
sparsity(df, tidy_up=True)

domain                                                                                                                                              contractaciopublica.gencat.cat  \
ContractFolderStatus ContractFolderID                                                                                                                                     1.000000   
                     ContractFolderStatusCode                                                                                                                             1.000000   
                     LegalDocumentReference     Attachment                          ExternalReference                      URI                                            0.353604   
                                                ID                                                                                                                        0.353604   
                     LocatedContractingParty    BuyerProfileURIID                                                                                                         1.000000   
                                                ParentLocatedParty                  ParentLocatedParty                     PartyName          Name                        0.000000   
                                                                                    PartyName                              Name                                           1.000000   
                                                Party                               PartyIdentification                    ID                                             0.000000   
                                                                                    PartyName                              Name                                           1.000000   
                     ProcurementProject         BudgetAmount                        EstimatedOverallContractAmount                                                        1.000000   
                                                                                    TaxExclusiveAmount                                                                    1.000000   
                                                Name                                                                                                                      1.000000   
                                                PlannedPeriod                       DurationMeasure                                                                       0.878378   
                                                                                    EndDate                                                                               0.121622   
                                                                                    StartDate                                                                             0.121622   
                                                RealizedLocation                    CountrySubentityCode                                                                  1.000000   
                                                RequiredCommodityClassification     ItemClassificationCode                                                                0.997748   
                                                TypeCode                                                                                                                  1.000000   
                     TechnicalDocumentReference Attachment                          ExternalReference                      URI                                            0.317568   
                                                ID                                                                                                                        0.317568   
                     TenderResult               AwardedTenderedProject              LegalMonetaryTotal                     TaxExclusiveAmount                             0.621622

If we set to `False` the `do_not_modify_input` parameter, then a new `domain` column is added to the **input** `pd.DataFrame`. By default, it's not there.

In [ ]:
'domain' in df.columns

False

In [ ]:
sparsity(df, do_not_modify_input=False)
'domain' in df.columns

True

In [ ]:
#| hide
from nbdev.doclinks import nbdev_export

In [ ]:
#| hide
nbdev_export('60_assemble.ipynb')